<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/multi_document_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Document Agents

In this guide, you learn towards setting up an agent that can effectively answer different types of questions over a larger set of documents.

These questions include the following

- QA over a specific doc
- QA comparing different docs
- Summaries over a specific doc
- Comparing summaries between different docs

We do this with the following architecture:

- setup a "document agent" over each Document: each doc agent can do QA/summarization within its doc
- setup a top-level agent over this set of document agents. Do tool retrieval and then do CoT over the set of tools to answer a question.

## Setup and Download Data

In this section, we'll define imports and then download Wikipedia articles about different cities. Each article is stored separately.

We load in 18 cities - this is not quite at the level of "hundreds" of documents but its still large enough to warrant some top-level document retrieval!

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index-agent-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 23.7 MB/s eta 0:00:00


In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

In [4]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

In [5]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [6]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

In [8]:

# city_docs

Define Global LLM and Embeddings

In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Building Multi-Document Agents

In this section we show you how to construct the multi-document agent. We first build a document agent for each document, and then define the top-level parent agent with an object index.

### Build Document Agent for each Document

In this section we define "document agents" for each document.

We define both a vector index (for semantic search) and summary index (for summarization) for each document. The two query engines are then converted into tools that are passed to an OpenAI function calling agent.

This document agent can dynamically choose to perform semantic search or summarization within a given document.

We create a separate document agent for each city.

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os
from llama_parse import LlamaParse
from llama_index.core.node_parser import MarkdownElementNodeParser
# node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []


# for idx, wiki_title in enumerate(wiki_titles):
#     nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])
#     all_nodes.extend(nodes)

#     if not os.path.exists(f"./data/{wiki_title}"):
#         # build vector index
#         vector_index = VectorStoreIndex(nodes)
#         vector_index.storage_context.persist(
#             persist_dir=f"./data/{wiki_title}"
#         )
#     else:
#         vector_index = load_index_from_storage(
#             StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
#         )
# Folder containing the PDF files
pdf_folder = '/content/pdf_folder'
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Initialize the parser
node_parser = MarkdownElementNodeParser(
    llm=Settings.llm,
    num_workers=8
)

all_nodes = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    documents = LlamaParse(result_type="markdown").load_data(pdf_path)

    nodes = node_parser.get_nodes_from_documents(documents)
    all_nodes.extend(nodes)

    # Use the PDF file name (without extension) as the identifier
    pdf_name = os.path.splitext(pdf_file)[0]

    if not os.path.exists(f"./data/{pdf_name}"):
        # Build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{pdf_name}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{pdf_name}"),
        )


    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {pdf_name} (e.g. policy key features, policy terms, premium payment mode, sum assured.)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {pdf_name}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    # function_llm = OpenAI(model="gpt-4")
    function_llm = Settings.llm
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {pdf_name}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[pdf_name] = agent
    query_engines[pdf_name] = vector_index.as_query_engine(
        similarity_top_k=2
    )

Started parsing the file under job_id cac11eca-04c3-47cb-8bc6-d02641fce0d4


1it [00:00, 1823.61it/s]
100%|██████████| 1/1 [00:01<00:00,  1.85s/it]
1it [00:00, 7025.63it/s]
100%|██████████| 1/1 [00:01<00:00,  1.81s/it]
3it [00:00, 19753.39it/s]
100%|██████████| 3/3 [00:01<00:00,  1.89it/s]
2it [00:00, 15709.00it/s]
100%|██████████| 2/2 [00:01<00:00,  1.01it/s]
1it [00:00, 8981.38it/s]
100%|██████████| 1/1 [00:01<00:00,  1.66s/it]
3it [00:00, 21583.04it/s]
100%|██████████| 3/3 [00:02<00:00,  1.03it/s]
4it [00:00, 7297.61it/s]
100%|██████████| 4/4 [00:01<00:00,  2.02it/s]
1it [00:00, 1598.44it/s]
100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 9000.65it/s]
100%|██████████| 1/1 [00:01<00:00,  1.64s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


Started parsing the file under job_id 08ad156e-b32c-419b-a6e5-82e6ff0e7fbe
.

1it [00:00, 9341.43it/s]
100%|██████████| 1/1 [00:01<00:00,  1.23s/it]
1it [00:00, 5637.51it/s]
100%|██████████| 1/1 [00:01<00:00,  1.29s/it]
1it [00:00, 1575.62it/s]
100%|██████████| 1/1 [00:01<00:00,  1.60s/it]
3it [00:00, 24244.53it/s]
100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


In [48]:
agents

{'hdfc-smart-term-insurence': <llama_index.agent.openai.base.OpenAIAgent at 0x7dbd30bbd2d0>,
 'max-life-smart-term-insurence': <llama_index.agent.openai.base.OpenAIAgent at 0x7dbd3002ea10>}

In [49]:
query_engines

{'hdfc-smart-term-insurence': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x7dbd30bbf100>,
 'max-life-smart-term-insurence': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x7dbd3002d030>}

In [52]:
all_nodes

[IndexNode(id_='f8e6790e-311a-4c78-8708-0145569429eb', embedding=None, metadata={'col_schema': 'Column: Life cover, till 75 years#.\nType: Text\nSummary: None\n\nColumn: Life cover, till 75 years#.\nType: Text\nSummary: None\n\nColumn: Life cover, till 75 years#.\nType: Text\nSummary: None'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='88405a55-9379-47e8-9a54-dc6786a03370', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='9d09030d5214727a5449e6af5500c4653e27d042583f3e6c51107464ad993375'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c4cbc4e5-53b1-4d68-9ae6-301247ffe4b1', node_type=<ObjectType.TEXT: '1'>, metadata={'table_df': '|Life cover, till 75 years#.|Life cover, till 75 years#.|Life cover, till 75 years#.|\n|---|\n|Return of premiums, up to 150%*.|Return of premiums, up to 150%*.|Return of premiums, up to 150%*.|\n|Upto 150%*|Life Cover|3 Plan|\n|return of prem

### Build Retriever-Enabled OpenAI Agent

We build a top-level agent that can orchestrate across the different document agents to answer any user query.

This agent takes in all document agents as tools. This specific agent `RetrieverOpenAIAgent` performs tool retrieval before tool use (unlike a default agent that tries to put all tools in the prompt).

Here we use a top-k retriever, but we encourage you to customize the tool retriever method!


In [13]:
pdf_files

['hdfc-smart-term-insurence.pdf', 'max-life-smart-term-insurence.pdf']

In [15]:
# define tool for each document agent
all_tools = []
for pdf_file in pdf_files:
    pdf_name = os.path.splitext(pdf_file)[0]
    pdf_summary = (
        f"This content contains policy details about {pdf_name}. Use"
        f" this tool if you want to answer any questions about {pdf_name}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[pdf_name],
        metadata=ToolMetadata(
            name=f"tool_{pdf_name}",
            description=pdf_summary,
        ),
    )
    all_tools.append(doc_tool)

In [51]:
all_tools

In [16]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [17]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about a set of policy documents.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

### Define Baseline Vector Store Index

As a point of comparison, we define a "naive" RAG pipeline which dumps all docs into a single vector index collection.

We set the top_k = 4

In [18]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

## Running Example Queries

Let's run some example queries, ranging from QA / summaries over a single document to QA / summarization over multiple documents.

In [19]:
# should use Boston agent -> vector tool
response = top_agent.query("Tell me about the hdfc term insurence policy?")

Added user message to memory: Tell me about the hdfc term insurence policy?
=== Calling Function ===
Calling function: tool_hdfc-smart-term-insurence with args: {"input":"overview"}
Added user message to memory: overview
=== Calling Function ===
Calling function: summary_tool with args: {"input":"hdfc-smart-term-insurence"}
Got output: HDFC Life Smart Term Edge is a Non-Linked, Non-Participating Individual Life Term Insurance Plan that offers coverage up to 75 years with the option for a return of premiums up to 150%. It provides different plan variants - Classic, Step-up, and Comprehensive, each with specific details and benefits. The plan allows for customization with additional riders like HDFC Life Accidental Death, Disability and Dismemberment Rider and HDFC Life Critical Illness Rider. Premium payment modes include monthly, half-yearly, and annual options. The plan also outlines terms and conditions related to grace period, lapse, reduced paid-up benefits, surrender value, free l

In [20]:
print(response)

The HDFC Life Smart Term Edge is a Non-Linked, Non-Participating Individual Life Term Insurance Plan that offers coverage up to 75 years with the option for a return of premiums up to 150%. It provides different plan variants - Classic, Step-up, and Comprehensive, each with specific details and benefits. The plan allows for customization with additional riders like HDFC Life Accidental Death, Disability and Dismemberment Rider and HDFC Life Critical Illness Rider. Premium payment modes include monthly, half-yearly, and annual options. The plan also outlines terms and conditions related to grace period, lapse, reduced paid-up benefits, surrender value, free look period, suicide clause, and revival policy. It is important to carefully review the policy terms and conditions for a comprehensive understanding of the associated risks and benefits before making a decision to purchase the plan.


In [21]:
# baseline
response = base_query_engine.query(
    "Tell me about the hdfc term insurence policy?"
)
print(str(response))

HDFC Life Smart Term Edge is a Non-Linked Non-Participating Individual Life Term Insurance Plan that offers different variants for desired protection, such as Classic, Step-up, and Comprehensive. It provides a substantial death benefit at affordable premiums and allows for customization with additional riders like Critical Illness and Accident Riders. The policy also offers a return of a percentage of premiums paid on completion of the policy term based on the variant chosen.


In [36]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "HDFC Life Critical Illness Rider can you explain this?"
)

Added user message to memory: HDFC Life Critical Illness Rider can you explain this?
=== Calling Function ===
Calling function: tool_hdfc-smart-term-insurence with args: {"input":"HDFC Life Critical Illness Rider"}
Added user message to memory: HDFC Life Critical Illness Rider
=== Calling Function ===
Calling function: summary_tool with args: {"input":"HDFC Life Critical Illness Rider"}
Got output: The HDFC Life Critical Illness Rider provides coverage for critical illnesses, where the rider sum assured is payable in a lump sum upon the diagnosis of any covered critical illness. The benefit amount is based on the terms and conditions of the rider, and it offers additional financial protection in case of such unfortunate occurrences.

Got output: The HDFC Life Critical Illness Rider provides coverage for critical illnesses, where the rider sum assured is payable in a lump sum upon the diagnosis of any covered critical illness. The benefit amount is based on the terms and conditions of t

In [38]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "what is the minimum sum assured for accident cover ?"
)

Added user message to memory: what is the minimum sum assured for accident cover ?
=== Calling Function ===
Calling function: tool_max-life-smart-term-insurence with args: {"input": "minimum sum assured for accident cover"}
Added user message to memory: minimum sum assured for accident cover
=== Calling Function ===
Calling function: vector_tool with args: {"input":"minimum sum assured for accident cover"}
Got output: The minimum sum assured for the accident cover is 100% of the Accident Cover Sum Assured, which will be payable as a lump sum in case the Life Insured dies due to an accident.

Got output: The minimum sum assured for accident cover is 100% of the Accident Cover Sum Assured, which will be payable as a lump sum in case the Life Insured dies due to an accident.

=== Calling Function ===
Calling function: tool_hdfc-smart-term-insurence with args: {"input": "minimum sum assured for accident cover"}
Added user message to memory: minimum sum assured for accident cover
=== Callin

In [39]:
print(response)

The minimum sum assured for accident cover is 100% of the Accident Cover Sum Assured in Max Life Smart Term Insurance, payable as a lump sum in case the Life Insured dies due to an accident. 

In HDFC Smart Term Insurance, the minimum sum assured for accident cover is ₹10,00,000.


In [42]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "what are different premium payment modes?"
)

Added user message to memory: what are different premium payment modes?
=== Calling Function ===
Calling function: tool_hdfc-smart-term-insurence with args: {"input": "premium payment modes"}
Added user message to memory: premium payment modes
=== Calling Function ===
Calling function: vector_tool with args: {"input":"premium payment modes"}
Got output: Premium payment modes vary based on the frequency of premium payments.

Got output: Premium payment modes for HDFC Smart Term Insurance vary based on the frequency of premium payments.

=== Calling Function ===
Calling function: tool_max-life-smart-term-insurence with args: {"input": "premium payment modes"}
Added user message to memory: premium payment modes
=== Calling Function ===
Calling function: vector_tool with args: {"input":"premium payment modes"}
Got output: Annual, Semi-Annual, Quarterly, Monthly premium payment modes.

Got output: The premium payment modes for Max Life Smart Term Insurance include Annual, Semi-Annual, Quart

In [44]:
print(response)

The premium payment modes for HDFC Smart Term Insurance vary based on the frequency of premium payments. For Max Life Smart Term Insurance, the premium payment modes include Annual, Semi-Annual, Quarterly, and Monthly payment options.


In [45]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "what are the differenc between both the plans ?"
)

Added user message to memory: what are the differenc between both the plans ?
=== Calling Function ===
Calling function: tool_hdfc-smart-term-insurence with args: {"input": "differences"}
Added user message to memory: differences
=== Calling Function ===
Calling function: summary_tool with args: {"input":"differences"}
Got output: The differences in the insurance plan variants include the type of protection offered, the percentage of return of premiums, and the additional benefits available. Each variant - Classic, Step-up, and Comprehensive - has unique features tailored to suit different protection needs and financial goals. The Classic variant provides protection with a return of premium, the Step-up variant offers higher return of premium, and the Comprehensive variant includes enhanced protection with the option for additional sum assured through Extra Protection. Additionally, the benefits payable on death or maturity vary between the variants based on the premiums paid and the p

In [46]:
print(response)

The differences between the HDFC Smart Term Insurance plan variants include the type of protection offered, the percentage of return of premiums, and the additional benefits available. Each variant - Classic, Step-up, and Comprehensive - has unique features tailored to suit different protection needs and financial goals. The Classic variant provides protection with a return of premium, the Step-up variant offers a higher return of premium, and the Comprehensive variant includes enhanced protection with the option for additional sum assured through Extra Protection. Additionally, the benefits payable on death or maturity vary between the variants based on the premiums paid and the policy terms chosen.

On the other hand, the Max Life Smart Term Plan offers various differences such as different Death Benefit Variants, Life Stage Add-On Sum Assured, Return of Premium with the Premium Back option, riders like Max Life Waiver of Premium Plus Rider and Accident Cover option, Accelerated Crit